# Penghindaran Tabrakan - Demo(Resnet18)

Dalam notebook ini kita akan menggunakan model yang kita latih untuk mendeteksi apakah robot ``free`` atau ``bloked`` untuk mengaktifkan perilaku menghindari tabrakan pada robot.

## Memuauat model yang telah terlatih

Kita akan berasumsi bahwa telah mengunduh ``best_model.pth`` ke workstation. Sekarang, Kita harus mengunggah model ini ke direktori buku catatan ini dengan menggunakan alat pengunggahan Jupyter Lab. Setelah selesai, seharusnya ada file bernama ``best_model.pth`` di direktori notebook ini.

> Pastikan file sudah terupload lengkap sebelum memanggil sel berikutnya

Jalankan kode di bawah ini untuk menginisialisasi model PyTorch. Ini akan terlihat sangat familiar dari buku catatan pelatihan.

In [1]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Selanjutnya, muat `trained weights` dari file ``best_model_resnet18.pth`` yang Anda unggah

In [2]:
model.load_state_dict(torch.load('best_model_resnet18.pth'))

<All keys matched successfully>

Saat ini, model `weights` yang terletak di memori CPU. Kita akan mengeksekusi kode di bawah ini untuk ditransfer ke perangkat GPU.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Membuat fungsi pra-pemrosesan

Kita sekarang telah memuat model, tetapi ada sedikit masalah. Format yang kita latih model kami tidak *persis* cocok dengan format kamera. Untuk melakukan itu, kita perlu melakukan beberapa *preprocessing*. Ini melibatkan langkah-langkah berikut:

1. Konversi dari tata letak HWC ke tata letak CHW
2. Normalisasi menggunakan parameter yang sama seperti yang kami lakukan selama pelatihan (kamera kami memberikan nilai dalam rentang [0, 255] dan gambar yang dimuat pelatihan dalam rentang [0, 1] sehingga kami perlu menskalakan 255,0
3. Transfer data dari memori CPU ke memori GPU
4. Tambahkan dimensi batch

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Kita sekarang telah mendefinisikan fungsi pra-pemrosesan yang dapat mengonversi gambar dari format kamera ke format input jaringan saraf.

Sekarang, mari kita mulai dan menampilkan kamera kita. Kita juga akan membuat penggeser yang akan menampilkan probabilitas bahwa robot diblokir. Kita juga akan menampilkan penggeser yang memungkinkan Kita mengontrol kecepatan dasar robot.

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

Kita juga akan membuat instance robot yang diperlukan untuk menggerakkan motor.

In [6]:
from jetbot import Robot

robot = Robot()

Selanjutnya, kita akan membuat fungsi yang akan dipanggil setiap kali nilai kamera berubah. Fungsi ini akan melakukan langkah-langkah berikut:

1. Pra-proses gambar kamera
2. Jalankan jaringan saraf
3. Sementara output jaringan saraf menunjukkan bahwa kita diblokir, kita akan belok kiri, jika tidak kita maju.

In [7]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
        
    # kita menerapkan fungsi `softmax` untuk menormalkan vektor output sehingga berjumlah 1 (yang menjadikannya distribusi probabilitas)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(speed_slider.value)
    else:
        robot.left(speed_slider.value)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # kita memanggil fungsi sekali untuk menginisialisasi

Keren! Kita telah membuat fungsi eksekusi jaringan saraf (Neural Network), tetapi sekarang Kita perlu melampirkannya ke kamera untuk diproses.

dengan melakukannya  fungsi ``observe``.

> PERINGATAN: Kode ini dapat memindahkan robot!! Sesuaikan penggeser kecepatan yang telah kita tentukan sebelumnya untuk mengontrol kecepatan robot dasar. bergerak cepat, jadi mulailah dengan lambat, dan tingkatkan nilainya secara bertahap.

In [8]:
camera.observe(update, names='value')  # ini melampirkan fungsi 'update' ke sifat 'value' kamera 

Luar biasa!
Jika Kita ingin menghentikan perilaku ini, Kita dapat melepaskan panggilan balik (callback) dengan menjalankan kode di bawah ini.

In [9]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # tambahkan sedikit 'sleep' untuk memastikan bingkai telah selesai diproses

robot.stop()

Mungkin Kita ingin robot berjalan tanpa streaming video ke browser. kita dapat memutuskan tautan kamera seperti di bawah ini.

In [ ]:
camera_link.unlink()  # jangan streaming ke browser (Ketika kamera hidup)

Sekali lagi, mari kita tutup koneksi kamera dengan benar agar kamera dapat digunakan di notebook lain.

In [ ]:
camera_link.link()  # streaming ke browser (tidak akan menjalankan kamera)